In [20]:
import os
import cv2
import string
import random
import glob
from PIL import Image, ImageChops
from claptcha import Claptcha

import numpy as np
import matplotlib.pyplot as plt
from shutil import copyfile

from matplotlib.pyplot import imshow
import numpy as np
from PIL import Image

%matplotlib inline


In [21]:
root_dir = os.getcwd()
image_output = os.path.join(root_dir,'generated_img_orig')
print(image_output)

image_clear_output = os.path.join(root_dir,'generated_img_clear')
print(image_clear_output)

image_clear_with_boxes_output = os.path.join(root_dir,'generated_img_clear_with_boxes')
print(image_clear_with_boxes_output)


/data/home/deep/notebooks/claptcha/generated_img_orig
/data/home/deep/notebooks/claptcha/generated_img_clear
/data/home/deep/notebooks/claptcha/generated_img_clear_with_boxes


In [22]:
def id_generator(size=6, chars=string.ascii_letters + string.digits):
    return ''.join(random.choice(chars) for _ in range(size))


In [23]:
classes = 'ABCDEFGHJKLMNPQRSTUVWXYZ23456789'
len(classes)

char_size_threshold = 20


In [24]:
# generating filename list - evenly!!!
RANGE1=list(range(0,16))
RANGE2=list(range(16,32))
RANGE3=list(range(9,25))
RANGE4=list(range(0,9))+list(range(25,32))

rnd_4letter_captcha = []
for l1 in RANGE1:
    l1name = classes[l1]
    for l2 in RANGE2:
        l2name =  classes[l2]
        for l3 in RANGE3:
            l3name =  classes[l3]
            for l4 in RANGE4:
                l4name =  classes[l4]
                concat_name = l1name + l2name + l3name +l4name
                rnd_4letter_captcha.append(concat_name)
print(len(rnd_4letter_captcha))

65536


In [25]:
for rnd_string in rnd_4letter_captcha:
    font = glob.glob(os.path.join(os.getcwd(),"fonts") + "/leadcoat.ttf")[0]
    #print('rnd char text = ' + rnd_string)
    c = Claptcha(rnd_string,font,margin=(10,100),
                 resample=Image.BICUBIC,size=(300, 300), noise=0.0)

    # creating array versions of clear and dirty images
    text,image_clear = c.clear_image
    text_for_dirty,image_dirty = c.dirty_image

    # creating file paths
    captcha_image_clear_file = os.path.join(image_clear_output, rnd_string +'.png')
    captcha_image_dirty_file = os.path.join(image_output,rnd_string + '.png')
    
    #saving clear image
    image_clear.save(captcha_image_clear_file)
    
    # generating bounding boxes using OpenCV contours
    captcha_image_file  = os.path.join(image_clear_output, rnd_string +'.png')        
    image = cv2.imread(captcha_image_file)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # threshold the image (convert it to pure black and white)
    thresh = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1]

    # find the contours (continuous blobs of pixels) the image
    contours = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Hack for compatibility with different OpenCV versions
    contours = contours[0] #if imutils.is_cv2() else contours[1]
    
    # removing false bouding boxes
    fixed_contours = []
    for contour in contours:
        # Get the rectangle that contains the contour
        (x, y, w, h) = cv2.boundingRect(contour)
        if w*h > char_size_threshold:
            fixed_contours.append(contour)
    
    # Now we can loop through each of the four contours and extract the letter
    # inside of each one
    letter_image_regions = []
    for contour in fixed_contours:    
        # Get the rectangle that contains the contour
        (x, y, w, h) = cv2.boundingRect(contour)
        
        # Compare the width and height of the contour to detect letters that
        # are conjoined into one chunk
        if w / h > 1.25:
            # This contour is too wide to be a single letter!
            # Split it in half into two letter regions!
            half_width = int(w / 2)
            letter_image_regions.append((x, y, half_width, h))
            letter_image_regions.append((x + half_width, y, half_width, h))
        else:
            # This is a normal letter by itself
            letter_image_regions.append((x, y, w, h))   
        
        ## create image label text file
    if len(letter_image_regions) != 4:
        ## delete image
        print('deleting image [{0}] due to num of bounding boxes diff than 4!'.format(captcha_image_clear_file))
        os.remove(captcha_image_clear_file)
        continue
    
    #sorting boxes accroding to pixel place
    letter_image_regions = sorted(letter_image_regions, key=lambda x: x[0])        
    
    #use for drawing boxes for later use in deubg
    clean_img_as_array = np.asarray(image_clear)
    dirty_img_as_array = np.asarray(image_dirty)

    # Save out each letter as a single image
    label = []
    for letter_bounding_box, letter_text in zip(letter_image_regions, rnd_string):
        # Grab the coordinates of the letter in the image
        x, y, w, h = letter_bounding_box
        (xmax,ymax , xmin , ymin)  = (x+w+1,y+h+1,x,y)

        # adding bounding boxes for debug
        clean_img_as_array = cv2.putText(clean_img_as_array, letter_text, (xmax,ymax), cv2.FONT_HERSHEY_SIMPLEX, 1.0,
                                         (255,0,0),lineType=cv2.LINE_AA) 
        clean_img_as_array = cv2.rectangle(clean_img_as_array,(xmax,ymax),(xmin,ymin),(0,255,0),3)

        dirty_img_as_array = cv2.putText(dirty_img_as_array, letter_text, (xmax,ymax), cv2.FONT_HERSHEY_SIMPLEX, 1.0,
                                         (255,0,0),lineType=cv2.LINE_AA) 
        dirty_img_as_array = cv2.rectangle(dirty_img_as_array,(xmax,ymax),(xmin,ymin),(0,255,0),3)        

        #print ("xmax {2} ymax {3} xmin {0} ymin {1}".format(xmin,ymin , xmax , ymax))
        print(','.join((str(xmin),str(ymin) , str(xmax) , str(ymax))))
        label.append(','.join((str(xmax),str(ymax) , str(xmin) , str(ymin))) + ',' + str(classes.index(letter_text) ))        
    
    # saving clear image with bounding box for debug
    im = Image.fromarray(clean_img_as_array)
    im.save(os.path.join(image_clear_with_boxes_output,rnd_string + ".png"))
    
    # saving annotation file
    print(os.path.join(image_clear_output, rnd_string+'.txt'))
    log_file = open(os.path.join(image_clear_output, rnd_string+'.txt'), 'a')
    for l in label:
          log_file.write(l +"\n") 
    log_file.close()
    copyfile(os.path.join(image_clear_output, rnd_string+'.txt'),
         os.path.join(image_output, rnd_string+'.txt'))
    
    # saving images as files
    image_dirty.save(captcha_image_dirty_file)
    
    
    
    